# 0.导包、导数据

In [115]:
import xgboost as xgb
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

from scipy.sparse import csc_matrix

from sklearn.ensemble import RandomForestClassifier  
from sklearn.decomposition import PCA
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn import preprocessing
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import log_loss

import warnings
warnings.filterwarnings('ignore')

In [99]:
# 取17-23作为测试，24作为验证
data = pd.read_pickle('data/round1_train2')
feature_train = data[data['f16'] <= 1537718400]
feature_train.drop('label',axis = 1,inplace = True)
feature_val = data[data['f16'] > 1537718400]
feature_val.drop('label',axis = 1,inplace = True)

label_train = data[data['f16'] <= 1537718400]['label']
label_val = data[data['f16'] > 1537718400]['label']

data = pd.read_pickle('data/round1_test2')
feature_test = data

data_index = pd.DataFrame(np.genfromtxt('data/round1_test.txt',dtype = np.str, delimiter=' ',skip_header=1))[0]

In [135]:
# 取18-23作为测试，24作为验证
data = pd.read_pickle('data/round1_train2')
feature_train = data[(data['f16'] <= 1537718400) & (data['f16'] > 1537200000)]
feature_train.drop('label',axis = 1,inplace = True)
feature_val = data[data['f16'] > 1537718400]
feature_val.drop('label',axis = 1,inplace = True)

label_train = data[(data['f16'] <= 1537718400) & (data['f16'] > 1537200000)]['label']
label_val = data[data['f16'] > 1537718400]['label']

data = pd.read_pickle('data/round1_test2')
feature_test = data

data_index = pd.DataFrame(np.genfromtxt('data/round1_test.txt',dtype = np.str, delimiter=' ',skip_header=1))[0]

In [149]:
# 取17-22作为测试，23作为验证
data = pd.read_pickle('data/round1_train2')
feature_train = data[data['f16'] <= 1537632000]
feature_train.drop('label',axis = 1,inplace = True)
feature_val = data[(data['f16'] > 1537632000) & (data['f16'] <= 1537718400)]
feature_val.drop('label',axis = 1,inplace = True)

label_train = data[data['f16'] <= 1537632000]['label']
label_val = data[(data['f16'] > 1537632000) & (data['f16'] <= 1537718400)]['label']

data = pd.read_pickle('data/round1_test2')
feature_test = data

data_index = pd.DataFrame(np.genfromtxt('data/round1_test.txt',dtype = np.str, delimiter=' ',skip_header=1))[0]

In [143]:
# 取18-22作为测试，23作为验证
data = pd.read_pickle('data/round1_train2')
feature_train = data[(data['f16'] <= 1537632000) & (data['f16'] > 1537200000)]
feature_train.drop('label',axis = 1,inplace = True)
feature_val = data[(data['f16'] > 1537632000) & (data['f16'] <= 1537718400)]
feature_val.drop('label',axis = 1,inplace = True)

label_train = data[(data['f16'] <= 1537632000) & (data['f16'] > 1537200000)]['label']
label_val = data[(data['f16'] > 1537632000) & (data['f16'] <= 1537718400)]['label']

data = pd.read_pickle('data/round1_test2')
feature_test = data

data_index = pd.DataFrame(np.genfromtxt('data/round1_test.txt',dtype = np.str, delimiter=' ',skip_header=1))[0]

In [129]:
feature_train.columns

Index(['f0', 'f1', 'f2:2_22731265849056483', 'f2:2_509660095530134768',
       'f2:2_1968056100269760729', 'f2:2_2011981573061447208',
       'f2:2_2436715285093487584', 'f2:2_2642175453151805566',
       'f2:2_3203673979138763595', 'f2:2_4879721024980945592',
       'f2:2_5755694407684602296', 'f2:2_5799347067982556520',
       'f2:2_7258015885215914736', 'f2:2_8277336076276184272',
       'f2:2_8710739180200009128', 'f2:3_-1', 'f2:3_6233669177166538628',
       'f2:3_8868887661186419229', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10',
       'f11:woman', 'f11:man', 'f11:family', 'f11:other', 'f12', 'f13:2002.0',
       'f13:2003.0', 'f13:2004.0', 'f13:2005.0', 'f14', 'f15', 'f16',
       'f16:day', 'f16:hour', 'f16:minute', 'f16:second', 'f16:dayofweek',
       'f16:dayofyear', 'f17', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25',
       'f1-ratio', 'f4-ratio', 'f5-ratio', 'f10-ratio', 'f19-ratio',
       'f1-f4-ratio', 'f1-f5-ratio', 'f4-f5-ratio', 'f4-f19-ratio',
       'f5-f19-ratio

In [65]:
feature_train.drop(['f1-f4-ratio', 'f1-f5-ratio', 'f4-f5-ratio', 'f4-f19-ratio','f5-f19-ratio'],axis = 1,inplace = True)

In [66]:
feature_val.drop(['f1-f4-ratio', 'f1-f5-ratio', 'f4-f5-ratio', 'f4-f19-ratio','f5-f19-ratio'],axis = 1,inplace = True)

In [67]:
feature_test.drop(['f1-f4-ratio', 'f1-f5-ratio', 'f4-f5-ratio', 'f4-f19-ratio','f5-f19-ratio'],axis = 1,inplace = True)

## 1.训练SVM

In [45]:
from sklearn import svm,datasets
from sklearn.metrics import log_loss

#调用SVC()
# 参数见：http://blog.csdn.net/xiaodongxiexie/article/details/70667101
clf = svm.SVC(
            C=0.001, 
            kernel='rbf', 
            degree=3, 
            gamma='auto', 
            coef0=0.0, 
            probability=False, 
            shrinking=True, 
            tol=0.001, 
            cache_size=200, 
            class_weight='balanced', 
            verbose=True, 
            max_iter=1000, 
            decision_function_shape='ovr', 
            random_state=None
)
#载入鸢尾花数据集
iris = datasets.load_iris()

#fit()训练
clf.fit(feature_train,label_train)

#predict()预测
label_val_svm = clf.predict(feature_val)
print(log_loss(label_val,label_val_svm))

[LibSVM]

/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


33.957631066


## 2.训练逻辑回归

In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss


feature_train_now = StandardScaler().fit_transform(feature_train)


clf_l1_LR= LogisticRegression(
#    tol = 0.0001,C = 0.00005, penalty = 'l1',max_iter = 500,class_weight = 'balanced',verbose = 2
) 
clf_l1_LR.fit(feature_train_now,label_train)
label_val_lr = clf_l1_LR.predict(feature_val)  
print(log_loss(label_val,label_val_lr))


clf_l2_LR= LogisticRegression(
#    tol = 0.0001,C = 0.0001, penalty = 'l2',max_iter = 500,class_weight = 'balanced',verbose = 2
) 
clf_l2_LR.fit(feature_train_now,label_train)
label_val_lr = clf_l2_LR.predict(feature_val)
print(log_loss(label_val,label_val_lr))

1.1711058309
1.1711058309


## 3.训练xgboost
这里要重点讲一下 Xgboost 的调参。通常认为对它性能影响较大的参数有：

- eta：每次迭代完成后更新权重时的步长。越小训练越慢。
- num_round：总共迭代的次数。
- subsample：训练每棵树时用来训练的数据占全部的比例。用于防止 Overfitting。
- colsample_bytree：训练每棵树时用来训练的特征的比例，类似 RandomForestClassifier 的 max_features。
- max_depth：每棵树的最大深度限制。与 Random Forest 不同，Gradient Boosting 如果不对深度加以限制，最终是会 Overfit 的。
- early_stopping_rounds：用于控制在 Out Of Sample 的验证集上连续多少个迭代的分数都没有提高后就提前终止训练。用于防止 Overfitting。

一般的调参步骤是：

- 将训练数据的一部分划出来作为验证集。
- 先将 eta 设得比较高（比如 0.1），num_round 设为 300 ~ 500。
- 用 Grid Search 对其他参数进行搜索
- 逐步将 eta 降低，找到最佳值。
- 以验证集为 watchlist，用找到的最佳参数组合重新在训练集上训练。注意观察算法的输出，看每次迭代后在验证集上分数的变化情况，从而得到最佳的 early_stopping_rounds。

In [150]:
train = xgb.DMatrix(feature_train,label = label_train)
val = xgb.DMatrix(feature_val,label = label_val)
test = xgb.DMatrix(feature_test)

In [151]:
# coding=utf-8


param = {
    # 1.General Parameters
    'booster': 'gbtree', # 提升计算的模型，可以是：gbtree, gblinear or dart
    'silent': 0,  # 0为打印信息，1为缄默
    # 'nthread': 4, # XGBoost运行时的线程数
    # 'num_pbuffer': # 预测缓冲区的大小，通常设置为训练实例的数量。缓冲区用于保存最后一次提升步骤的预测结果。
    # 'num_feature': # 特征值数量
    
    # 2.Booster Parameters
    'eta': 0.05,  # 学习率，默认0.3，取值范围为：[0,1]，典型值为0.01-0.2，越小越保守
    'gamma': 0.1,  # 节点分裂所需的最小损失函数下降值，和损失函数息息相关。默认0，典型值0.1、0.2，越大越保守，
    'max_depth': 5,  # 树的最大深度，默认6，典型值为3-10，越大越易过拟合。
    'min_child_weight':1, # 这个参数非常影响结果，最小叶子节点样本权重和。如果一个叶子节点的样本权重和小于min_child_weight则拆分过程结束。在现行回归模型中，这个参数是指建立每个模型所需要的最小样本数。默认1，越大算法越易欠拟合
    'max_delta_step ':0, # 限制每棵树权重改变的最大步长。越大越保守。通常，这个参数不需要设置。但是当各类别的样本十分不平衡时，它对逻辑回归是很有帮助的。
    'subsample':0.8, # 用于训练模型的子样本占整个样本集合的比例，行采样，典型值0.5-1，越小越保守。
    'colsample_bytree ': 0.8,  # 在建立树时对特征采样的比例，列采样。典型值0.5-1

    'lambda':10, # 权重的L2正则化项，默认是1
    'alpha':1,# 权重的L1正则化项，默认是1
    'lambda_bias': 0,  # 在偏置上的L2正则。缺省值为0（在L1上没有偏置项的正则，因为L1时偏置不重要）
    # 'scale_pos_weight':0.5, # 各类样本十分不平衡时，把这个参数设置为一个正数，可以使算法更快收敛。默认是1

    # 3.Task Parameters
    # 'tree_method': 'approx',
    'objective': 'binary:logistic',  # 使用的模型，分类的数目
    'eval_metric':'logloss', # 校验数据所需要的评价指标，不同的目标函数将会有缺省的评价指标
    'base_score':0.5, # 对于所有样本预测为正样本的全局偏置（初始分数）。如果迭代次数够多，改变这个参数对结果不会有影响。
    'seed':0 # 随机数的种子
}
num_boost_round = 1000  # 迭代的次数，弱分类器的数量

watchlist = [(train, 'train'),(val, 'eval')]  # 看板，每次迭代都可以在控制台打印出训练集与测试集的损失

In [152]:
# obj、feval、early_stopping_rounds、evals_result、verbose_eval、xgb_model

bst = xgb.train(param, train, num_boost_round, evals=watchlist, obj=None, feval=None, maximize=False,
      early_stopping_rounds=200, evals_result=None, verbose_eval=True, xgb_model=None, callbacks=None,learning_rates=None)
bst.save_model('data/model_xgboost')

[0]	train-logloss:0.648113	eval-logloss:0.647871
Multiple eval metrics have been passed: 'eval-logloss' will be used for early stopping.

Will train until eval-logloss hasn't improved in 200 rounds.
[1]	train-logloss:0.607363	eval-logloss:0.606894
[2]	train-logloss:0.570336	eval-logloss:0.569654
[3]	train-logloss:0.536558	eval-logloss:0.53567
[4]	train-logloss:0.505631	eval-logloss:0.504551
[5]	train-logloss:0.477269	eval-logloss:0.476002
[6]	train-logloss:0.451169	eval-logloss:0.449803
[7]	train-logloss:0.427103	eval-logloss:0.425585
[8]	train-logloss:0.404849	eval-logloss:0.403162
[9]	train-logloss:0.384252	eval-logloss:0.382428
[10]	train-logloss:0.365151	eval-logloss:0.363163
[11]	train-logloss:0.347418	eval-logloss:0.345277
[12]	train-logloss:0.330933	eval-logloss:0.328646
[13]	train-logloss:0.315595	eval-logloss:0.31316
[14]	train-logloss:0.301291	eval-logloss:0.298736
[15]	train-logloss:0.287951	eval-logloss:0.285266
[16]	train-logloss:0.275497	eval-logloss:0.27269
[17]	train-lo

[161]	train-logloss:0.088855	eval-logloss:0.083508
[162]	train-logloss:0.088834	eval-logloss:0.083506
[163]	train-logloss:0.088808	eval-logloss:0.083513
[164]	train-logloss:0.088785	eval-logloss:0.083512
[165]	train-logloss:0.088764	eval-logloss:0.083508
[166]	train-logloss:0.088747	eval-logloss:0.083509
[167]	train-logloss:0.088723	eval-logloss:0.083503
[168]	train-logloss:0.088698	eval-logloss:0.083498
[169]	train-logloss:0.08868	eval-logloss:0.083491
[170]	train-logloss:0.088655	eval-logloss:0.083496
[171]	train-logloss:0.088631	eval-logloss:0.083492
[172]	train-logloss:0.088618	eval-logloss:0.083489
[173]	train-logloss:0.088591	eval-logloss:0.083483
[174]	train-logloss:0.088576	eval-logloss:0.083479
[175]	train-logloss:0.088556	eval-logloss:0.083472
[176]	train-logloss:0.08854	eval-logloss:0.083471
[177]	train-logloss:0.08852	eval-logloss:0.083469
[178]	train-logloss:0.088496	eval-logloss:0.083466
[179]	train-logloss:0.088473	eval-logloss:0.083463
[180]	train-logloss:0.08845	eval-l

[323]	train-logloss:0.085802	eval-logloss:0.083448
[324]	train-logloss:0.085786	eval-logloss:0.08346
[325]	train-logloss:0.085777	eval-logloss:0.083459
[326]	train-logloss:0.085757	eval-logloss:0.083462
[327]	train-logloss:0.085745	eval-logloss:0.083462
[328]	train-logloss:0.085726	eval-logloss:0.083461
[329]	train-logloss:0.085703	eval-logloss:0.083462
[330]	train-logloss:0.085684	eval-logloss:0.083464
[331]	train-logloss:0.085673	eval-logloss:0.083468
[332]	train-logloss:0.085662	eval-logloss:0.083469
[333]	train-logloss:0.085647	eval-logloss:0.083468
[334]	train-logloss:0.085638	eval-logloss:0.083465
[335]	train-logloss:0.085623	eval-logloss:0.083466
[336]	train-logloss:0.0856	eval-logloss:0.083478
[337]	train-logloss:0.085584	eval-logloss:0.083475
[338]	train-logloss:0.085568	eval-logloss:0.083489
[339]	train-logloss:0.085543	eval-logloss:0.083483
[340]	train-logloss:0.085528	eval-logloss:0.083478
[341]	train-logloss:0.085505	eval-logloss:0.08348
[342]	train-logloss:0.085492	eval-l

In [111]:
importance = bst.get_fscore()

In [112]:
importance = sorted(importance,key=lambda s: s[1])

In [113]:
importance

['f 2 : 2 _ 7 2 5 8 0 1 5 8 8 5 2 1 5 9 1 4 7 3 6',
 'f 1 0 - r a t i o',
 'f 1 9',
 'f 1 - f 5 - r a t i o',
 'f 2 : 3 _ 6 2 3 3 6 6 9 1 7 7 1 6 6 5 3 8 6 2 8',
 'f 0',
 'f 4 - f 5 - r a t i o',
 'f 6',
 'f 1 - c o u n t',
 'f 1 - r a t i o',
 'f 4 - r a t i o',
 'f 1 5',
 'f 1 6',
 'f 1',
 'f 1 0',
 'f 2 4',
 'f 1 - f 1 0 - b r o w s e - c o u n t',
 'f 5 - r a t i o',
 'f 1 9 - r a t i o',
 'f 5',
 'f 5 - f 1 9 - r a t i o',
 'f 4 - f 1 9 - r a t i o',
 'f 4',
 'f 1 4',
 'f 5 - f 1 0 - b r o w s e - c o u n t',
 'f 4 - f 5 - b r o w s e - c o u n t',
 'f 2 1',
 'f 2 3',
 'f 2 0',
 'f 1 - b r o w s e - c o u n t',
 'f 4 - f 5 - c o u n t',
 'f 2 : 3 _ 8 8 6 8 8 8 7 6 6 1 1 8 6 4 1 9 2 2 9',
 'f 2 5',
 'f 1 2',
 'f 1 6 : s e c o n d',
 'f 2 : 2 _ 4 8 7 9 7 2 1 0 2 4 9 8 0 9 4 5 5 9 2',
 'f 1 6 : m i n u t e',
 'f 1 0 - c o u n t',
 'f 1 7',
 'f 1 0 - b r o w s e - c o u n t',
 'f 2 : 2 _ 1 9 6 8 0 5 6 1 0 0 2 6 9 7 6 0 7 2 9',
 'f 1 6 : h o u r',
 'f 7',
 'f 2 : 2 _ 2 6 4 2 1 7 5 4 5 

In [153]:
# 预测
label_test_xgb = bst.predict(test)

In [154]:
# 4.保存
np.savetxt('data/round1_result_xgb.txt', np.c_[data_index, label_test_xgb], delimiter=',', header='instance_id predicted_score',comments='', fmt='%s %f')

# 4.AdaBoost

In [55]:
clf = AdaBoostClassifier() #迭代100次  
clf.fit(feature_train,label_train)
label_val_lr = clf.predict(feature_val)  
print(log_loss(label_val,label_val_lr))

0.581931454242


# 5.训练Random Forest

In [56]:
clf = RandomForestClassifier()
clf.fit(feature_train , label_train)
label_val_rf = clf.predict(feature_val)    
print(log_loss(label_val,label_val_rf))

0.584341164082


# 6.lightbgm

In [97]:
import lightgbm as lgb

gbm = lgb.LGBMRegressor(objective='binary',

                        num_leaves=64,

                        learning_rate=0.01,

                        n_estimators=2000)

gbm.fit(feature_train, label_train,

        eval_set=[(feature_val, label_val)],

        eval_metric='binary_logloss',

        early_stopping_rounds=300)

[1]	valid_0's binary_logloss: 0.0855711
Training until validation scores don't improve for 300 rounds.
[2]	valid_0's binary_logloss: 0.0855023
[3]	valid_0's binary_logloss: 0.085435
[4]	valid_0's binary_logloss: 0.0853771
[5]	valid_0's binary_logloss: 0.0853116
[6]	valid_0's binary_logloss: 0.0852492
[7]	valid_0's binary_logloss: 0.0851836
[8]	valid_0's binary_logloss: 0.0851136
[9]	valid_0's binary_logloss: 0.085054
[10]	valid_0's binary_logloss: 0.0849983
[11]	valid_0's binary_logloss: 0.0849607
[12]	valid_0's binary_logloss: 0.0849113
[13]	valid_0's binary_logloss: 0.0848758
[14]	valid_0's binary_logloss: 0.0848263
[15]	valid_0's binary_logloss: 0.0847919
[16]	valid_0's binary_logloss: 0.0847416
[17]	valid_0's binary_logloss: 0.0846964
[18]	valid_0's binary_logloss: 0.0846587
[19]	valid_0's binary_logloss: 0.0846124
[20]	valid_0's binary_logloss: 0.0845807
[21]	valid_0's binary_logloss: 0.084534
[22]	valid_0's binary_logloss: 0.0845046
[23]	valid_0's binary_logloss: 0.0844671
[24]	v

[199]	valid_0's binary_logloss: 0.0823638
[200]	valid_0's binary_logloss: 0.0823614
[201]	valid_0's binary_logloss: 0.0823569
[202]	valid_0's binary_logloss: 0.082353
[203]	valid_0's binary_logloss: 0.0823513
[204]	valid_0's binary_logloss: 0.0823463
[205]	valid_0's binary_logloss: 0.082346
[206]	valid_0's binary_logloss: 0.0823319
[207]	valid_0's binary_logloss: 0.0823313
[208]	valid_0's binary_logloss: 0.0823276
[209]	valid_0's binary_logloss: 0.0823253
[210]	valid_0's binary_logloss: 0.0823281
[211]	valid_0's binary_logloss: 0.0823262
[212]	valid_0's binary_logloss: 0.0823262
[213]	valid_0's binary_logloss: 0.0823235
[214]	valid_0's binary_logloss: 0.0823169
[215]	valid_0's binary_logloss: 0.0823192
[216]	valid_0's binary_logloss: 0.0823189
[217]	valid_0's binary_logloss: 0.0823165
[218]	valid_0's binary_logloss: 0.0823126
[219]	valid_0's binary_logloss: 0.0823101
[220]	valid_0's binary_logloss: 0.082308
[221]	valid_0's binary_logloss: 0.0823046
[222]	valid_0's binary_logloss: 0.082

[407]	valid_0's binary_logloss: 0.0819869
[408]	valid_0's binary_logloss: 0.0819882
[409]	valid_0's binary_logloss: 0.0819894
[410]	valid_0's binary_logloss: 0.0819897
[411]	valid_0's binary_logloss: 0.0819925
[412]	valid_0's binary_logloss: 0.0819932
[413]	valid_0's binary_logloss: 0.0819915
[414]	valid_0's binary_logloss: 0.0819932
[415]	valid_0's binary_logloss: 0.0819934
[416]	valid_0's binary_logloss: 0.0819946
[417]	valid_0's binary_logloss: 0.0819972
[418]	valid_0's binary_logloss: 0.0819982
[419]	valid_0's binary_logloss: 0.0819989
[420]	valid_0's binary_logloss: 0.0819967
[421]	valid_0's binary_logloss: 0.0819976
[422]	valid_0's binary_logloss: 0.0819983
[423]	valid_0's binary_logloss: 0.082004
[424]	valid_0's binary_logloss: 0.0820062
[425]	valid_0's binary_logloss: 0.0820058
[426]	valid_0's binary_logloss: 0.0820077
[427]	valid_0's binary_logloss: 0.0820152
[428]	valid_0's binary_logloss: 0.0820161
[429]	valid_0's binary_logloss: 0.082015
[430]	valid_0's binary_logloss: 0.08

[608]	valid_0's binary_logloss: 0.0820889
[609]	valid_0's binary_logloss: 0.0820916
[610]	valid_0's binary_logloss: 0.0820937
[611]	valid_0's binary_logloss: 0.0820942
[612]	valid_0's binary_logloss: 0.0820928
[613]	valid_0's binary_logloss: 0.0820861
[614]	valid_0's binary_logloss: 0.0820902
[615]	valid_0's binary_logloss: 0.0820974
[616]	valid_0's binary_logloss: 0.082097
[617]	valid_0's binary_logloss: 0.0821001
[618]	valid_0's binary_logloss: 0.0820982
[619]	valid_0's binary_logloss: 0.0820999
[620]	valid_0's binary_logloss: 0.0821003
[621]	valid_0's binary_logloss: 0.0821015
[622]	valid_0's binary_logloss: 0.082104
[623]	valid_0's binary_logloss: 0.0821045
[624]	valid_0's binary_logloss: 0.0821045
[625]	valid_0's binary_logloss: 0.082108
[626]	valid_0's binary_logloss: 0.082111
[627]	valid_0's binary_logloss: 0.0821212
[628]	valid_0's binary_logloss: 0.0821229
[629]	valid_0's binary_logloss: 0.0821217
[630]	valid_0's binary_logloss: 0.0821222
[631]	valid_0's binary_logloss: 0.0821

LGBMRegressor(boosting_type='gbdt', colsample_bytree=1.0, learning_rate=0.01,
       max_bin=255, max_depth=-1, min_child_samples=10, min_child_weight=5,
       min_split_gain=0.0, n_estimators=2000, n_jobs=-1, num_leaves=64,
       objective='binary', random_state=0, reg_alpha=0.0, reg_lambda=0.0,
       silent=True, subsample=1.0, subsample_for_bin=50000,
       subsample_freq=1)

In [63]:
label_val_bgm = gbm.predict(feature_val, num_iteration=gbm.best_iteration_)

label_test = gbm.predict(feature_test, num_iteration=gbm.best_iteration_)

print(log_loss(label_val,label_val_bgm))

0.0821183476381


In [64]:
np.savetxt('data/round1_result_lgb.txt', np.c_[data_index, label_test], delimiter=',', header='instance_id predicted_score',comments='', fmt='%s %f')

# 7.GBDT

In [61]:
from sklearn.ensemble import GradientBoostingClassifier    
model = GradientBoostingClassifier()    
model.fit(feature_train, label_train)    
label_val_gbdt = clf.predict(feature_val)  
print(log_loss(label_val,label_val_gbdt))

0.584341164082


# 8.FM/FFM


In [62]:
label_train[label_train < 0.5] = -1.0

In [63]:
feature_train = csc_matrix(feature_train)

In [64]:
feature_val = csc_matrix(feature_val)

In [65]:
from fastFM import sgd,mcmc
fm = sgd.FMClassification(
#     n_iter=1000, init_stdev=0.1, l2_reg_w=0,l2_reg_V=0, rank=2, step_size=0.1
)
fm.fit(feature_train, label_train)
label_val_fm = fm.predict(feature_val)

# fm = mcmc.FMClassification(n_iter=1000, rank=2, init_stdev=0.1)
# y_pred = fm.fit_predict(X_train, y_train, X_test)
# label_val_fm = fm.fit_predict_proba(X_train, y_train, X_test)

/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/opt/conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1735: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0
/opt/conda/lib/python3.6/site-packages/fastFM/base.py:123: RuntimeWarning: invalid value encountered in greater
  y_pred[y_proba > .5] = self.classes_[1]


In [66]:
pd.Series(label_val_fm).value_counts()

-1.0    57334
dtype: int64

# 9.集成
要求：
- Base Model 之间的相关性要尽可能的小
- Base Model 之间的性能表现不能差距太大

## 9.1Bagging
- 使用训练数据的不同随机子集来训练每个 Base Model，最后进行每个 Base Model 权重相同的 Vote。

## 9.2Boosting
- 迭代地训练 Base Model，每次根据上一个迭代中预测错误的情况修改训练样本的权重。

## 9.3Blending
- 用不相交的数据训练不同的 Base Model，将它们的输出取（加权）平均。实现简单，但对训练数据利用少了。

## 9.4Stacking
- 我用了两层的模型融合，Level 1使用了：XGBoost、LightGBM、RandomForest、ExtraTrees、DecisionTree、AdaBoost，一共6个模型，Level 2使用了LinearRegression来拟合第一层的结果。
- http://blog.csdn.net/a358463121/article/details/53054686